# Setup

In [18]:
# SQL Imports
import sqlite3
import csv

# Data Wrangling
import pandas as pd

# Loading Data

In [19]:
sql_connection = sqlite3.connect("../data/raw/spacextbl.db")
sql_cursor = sql_connection.cursor()

In [20]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql(
    "SPACEXTBL", 
    sql_connection, 
    if_exists="replace", 
    index=False,
    method="multi"
)

101

In [21]:
%load_ext sql
%sql sqlite:///../data/raw/spacextbl.db
%sql CREATE TABLE IF NOT EXISTS SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:///../data/raw/spacextbl.db
Done.


[]

# Exploring Data

##### Display the names of the unique launch sites  in the space mission

In [22]:
%sql SELECT DISTINCT Launch_Site FROM spacextable

 * sqlite:///../data/raw/spacextbl.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


4 Launch Sites were identified

#####  Display 5 records where launch sites begin with the string 'CCA' 

In [23]:
%sql SELECT * FROM spacextable WHERE Launch_Site LIKE "CCA%" LIMIT 5

 * sqlite:///../data/raw/spacextbl.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


LIKE function was used to identify launch sites starting with ‘CCA’.

The returned table was limited to five rows

##### Display the total payload mass carried by boosters launched by NASA (CRS)

In [24]:
%sql SELECT SUM(PAYLOAD_MASS__KG_) FROM spacextable WHERE Customer = "NASA (CRS)"

 * sqlite:///../data/raw/spacextbl.db
Done.


SUM(PAYLOAD_MASS__KG_)
45596


45.5 tons of Payload Mass was carried by boosters launched by NASA (CRS).

##### Display average payload mass carried by booster version F9 v1.1

In [25]:
%sql SELECT AVG(PAYLOAD_MASS__KG_) FROM spacextable WHERE Booster_Version LIKE "F9 v1.1%"

 * sqlite:///../data/raw/spacextbl.db
Done.


AVG(PAYLOAD_MASS__KG_)
2534.6666666666665


2.5 tons is the average Payload Mass of F9 v1.1 boosters.

The query included all F9 v1.1 models.

##### List the date when the first succesful landing outcome in ground pad was acheived.

In [26]:
%sql SELECT MIN(Date) FROM spacextable WHERE Landing_Outcome = "Success (ground pad)"

 * sqlite:///../data/raw/spacextbl.db
Done.


MIN(Date)
2015-12-22


First success in ground pad was achieved in 2015-12-22.

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [27]:
%sql SELECT Booster_Version, COUNT(*) AS occurrences \
FROM spacextable \
WHERE PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000 AND Landing_Outcome = "Success (drone ship)" \
GROUP BY Booster_Version

 * sqlite:///../data/raw/spacextbl.db
Done.


Booster_Version,occurrences
F9 FT B1021.2,1
F9 FT B1031.2,1
F9 FT B1022,1
F9 FT B1026,1


The four boosters listed in the result of the query had a success in drone ship on payloads between 4k and 6k, with one occurrence each.

##### List the total number of successful and failure mission outcomes

In [28]:
%sql SELECT Mission_Outcome, COUNT(*) AS occurrences \
FROM spacextable \
GROUP BY Mission_Outcome

 * sqlite:///../data/raw/spacextbl.db
Done.


Mission_Outcome,occurrences
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


One mission failed during flight. The rest are all successful.

Data could be further processed to binarize the categories in “Failure” or “Success” only.

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [29]:
%sql SELECT Booster_Version \
FROM spacextable \
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM spacextable)

 * sqlite:///../data/raw/spacextbl.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


Subquery was used to get the maximum payload mass.

12 boosters carried its maximum.

##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**

In [30]:
%sql SELECT substr(Date, 6, 2) as Month_Name, Booster_version, Launch_Site, Landing_Outcome \
FROM SPACEXTBL \
WHERE Landing_Outcome = "Failure (drone ship)" and SUBSTR(Date, 0, 5)= "2015"

 * sqlite:///../data/raw/spacextbl.db
Done.


Month_Name,Booster_Version,Launch_Site,Landing_Outcome
01,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
04,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


Two failures on drone shipping were recorded in 2015, both on CCAFS LC-40.

##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [31]:
%sql SELECT Landing_Outcome, COUNT(*) as occurrences \
FROM SPACEXTBL \
WHERE Date BETWEEN "2010-06-04" AND "2017-03-20" \
GROUP BY Landing_Outcome \
ORDER BY occurrences DESC

 * sqlite:///../data/raw/spacextbl.db
Done.


Landing_Outcome,occurrences
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


GROUP BY and ORDER BY clauses were used to get the results.

Top 1 is not-attempted landings.